# **GOAL**: To Build a Sarcasm Classfier using ML and NLP 
**Problem description**:-
We have given the News Headlines Dataset and our goal is to predict whether a given text is sarcastic or not.

**Dataset**:-This News Headlines Dataset for Sarcasm Detection is collected from The Onion website which aims at producing sarcastic versions of current events and HuffPost website which collects real news headlines.
- Download the dataset [here.](https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection)

Useful Blogs:
- https://monkeylearn.com/blog/text-cleaning/
- https://www.geeksforgeeks.org/python-stemming-words-with-nltk/




In [8]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [9]:
pip install pyforest

Note: you may need to restart the kernel to use updated packages.


**Intro about NLTK(Natural Language Toolkit)**:

NLTK is a platform used for building Python programs that work with human language data for applying in statistical natural language processing (NLP). It contains text processing libraries for tokenization, parsing, classification, stemming, tagging and semantic reasoning.

In [10]:
import pyforest #to import all the python libraries at once
import json
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download("wordnet")
import nltk.corpus
from nltk import punkt
from nltk import wordnet
from nltk.corpus import stopwords
import time

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rahulmullaguru/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rahulmullaguru/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rahulmullaguru/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
#to read the data in json file and creating a d'frame out of it
data=pd.read_json("Sarcasm_Headlines_Dataset.json", lines=True)

# For Rahul Narang-- Drive location
#data= pd.read_json("/content/drive/MyDrive/Sarcasm_Headlines_Dataset.json", lines=True)
#data.head(10)

<IPython.core.display.Javascript object>

In [12]:
print(data.isnull().any(axis=0)) #check for any null values

article_link    False
headline        False
is_sarcastic    False
dtype: bool


In [13]:
import re
#as we observed some special symbols in the text under "headline"., we removed them using library "re"
data['headline'] = data['headline'].map(lambda x: re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", ' ', x))
data.head(20)
#/^[a-zA-Z0-9!@#$%^&*()_+\-=\[\]{};':"\\|,.<>\/?]*$/

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the roseanne revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son s web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,boehner just wants wife to listen not come up...,1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j k rowling wishes snape happy birthday in th...,0
5,https://www.huffingtonpost.com/entry/advancing...,advancing the world s women,0
6,https://www.huffingtonpost.com/entry/how-meat-...,the fascinating case for eating lab grown meat,0
7,https://www.huffingtonpost.com/entry/boxed-col...,this ceo will send your kids to school if you...,0
8,https://politics.theonion.com/top-snake-handle...,top snake handler leaves sinking huckabee camp...,1
9,https://www.huffingtonpost.com/entry/fridays-m...,friday s morning email inside trump s presser...,0


In [14]:
#renaming the "headline" column as "text"
data=data.rename(columns={"headline":"text"})
data

,article_link,text,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the roseanne revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son s web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,boehner just wants wife to listen not come up...,1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j k rowling wishes snape happy birthday in th...,0
...,...,...,...
26704,https://www.huffingtonpost.com/entry/american-...,american politics in moral free fall,0
26705,https://www.huffingtonpost.com/entry/americas-...,america s best 20 hikes,0
26706,https://www.huffingtonpost.com/entry/reparatio...,reparations and obama,0
26707,https://www.huffingtonpost.com/entry/israeli-b...,israeli ban targeting boycott supporters raise...,0


In [15]:
text = data["text"]
is_sarcastic = data["is_sarcastic"]

In [16]:
text = text.apply(lambda x: x.lower()) #axis=1 applies to each row
text

0        former versace store clerk sues over secret  b...
1        the  roseanne  revival catches up to our thorn...
2        mom starting to fear son s web series closest ...
3        boehner just wants wife to listen  not come up...
4        j k  rowling wishes snape happy birthday in th...
                               ...                        
26704                 american politics in moral free fall
26705                              america s best 20 hikes
26706                                reparations and obama
26707    israeli ban targeting boycott supporters raise...
26708                    gourmet gifts for the foodie 2014
Name: text, Length: 26709, dtype: object

In [17]:
stop = stopwords.words('english')
text = text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [18]:
print(data.shape)
print(data['is_sarcastic'].value_counts())
data[data["is_sarcastic"]==0].head(5)

(26709, 3)
0    14985
1    11724
Name: is_sarcastic, dtype: int64


,article_link,text,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the roseanne revival catches up to our thorn...,0
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j k rowling wishes snape happy birthday in th...,0
5,https://www.huffingtonpost.com/entry/advancing...,advancing the world s women,0
6,https://www.huffingtonpost.com/entry/how-meat-...,the fascinating case for eating lab grown meat,0


Preprocessing Steps:
1. Stemming
2. Vectorization 
3. Tokenization
4. Lemmatization

## Stemming 
Stemming is the process of reducing a word to its word stem that affixes to suffixes and prefixes or the roots of words known as a lemma. Stemming is important in natural language understanding (NLU) and natural language processing (NLP).
for example:-
original word: “reading” and “reader”.
after stemming: read (for all the above two words).

In [19]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps = PorterStemmer()
from nltk.tokenize import word_tokenize

In [20]:
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(ps.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [21]:
textstem = text.apply(lambda x: stemSentence(x))

In [22]:
textstem

0        former versac store clerk sue secret black cod...
1        roseann reviv catch thorni polit mood better w...
2        mom start fear son web seri closest thing gran...
3        boehner want wife listen come altern debt redu...
4            j k rowl wish snape happi birthday magic way 
                               ...                        
26704                      american polit moral free fall 
26705                                america best 20 hike 
26706                                         repar obama 
26707    isra ban target boycott support rais alarm abr...
26708                             gourmet gift foodi 2014 
Name: text, Length: 26709, dtype: object

```
#alternative
text = text.apply(lambda x: x.split())
text = text.apply(lambda x :" ".join([ps.stem(word) for word in x]))
```



## Lemmatization  
Lemmatization, on the other hand, groups words based on root definition, and allows us to differentiate between present, past, and indefinite.
So, ‘jumps’ and ‘jump’ are grouped into the present ‘jump’, as different from all uses of ‘jumped’ which are grouped together as past tense, and all instances of ‘jumping’ which are grouped together as the indefinite (meaning continuing/continuous).

**Note:** 
In general, lemmatization offers better precision than stemming, but at the expense of recall. As we've seen, stemming and lemmatization are effective techniques to expand recall, with lemmatization giving up some of that recall to increase precision.

In [23]:
lm = WordNetLemmatizer()

In [24]:
def LemmSentence(sentence):
    token_words=word_tokenize(sentence)
    lemm_sentence=[]
    for word in token_words:
        lemm_sentence.append(lm.lemmatize(word))
        lemm_sentence.append(" ")
    return "".join(lemm_sentence)

In [25]:
textlemm = text.apply(lambda x: LemmSentence(x))
textlemm

0        former versace store clerk sue secret black co...
1        roseanne revival catch thorny political mood b...
2        mom starting fear son web series closest thing...
3        boehner want wife listen come alternative debt...
4        j k rowling wish snape happy birthday magical ...
                               ...                        
26704                   american politics moral free fall 
26705                                america best 20 hike 
26706                                    reparation obama 
26707    israeli ban targeting boycott supporter raise ...
26708                            gourmet gift foodie 2014 
Name: text, Length: 26709, dtype: object

In [26]:
data['textlemm'] = textlemm

In [27]:
textstem [:3]

0    former versac store clerk sue secret black cod...
1    roseann reviv catch thorni polit mood better w...
2    mom start fear son web seri closest thing gran...
Name: text, dtype: object

## Vectorization:
**Some popular methods to accomplish text vectorization:**

* Binary Term Frequency.
* Bag of Words (BoW) Term Frequency.
* (L1) Normalized Term Frequency.
* (L2) Normalized TF-IDF.
* Word2Vec.

Corpus - Collection of texts

In [28]:
# Text Vectorization
# Text Vectorization is the process of converting text into numerical representation
# In the NLP world, it is known as Word embeddings.
#Text Vectorization
# Text Vectorization is the process of converting text into numerical representation
# In the NLP world, it is known as word embeddings.
##Link: https://www.analyticsvidhya.com/blog/2021/06/part-5-step-by-step-guide-to-master-nlp-text-vectorization-approaches/
#Term frequency-inverse document frequency ( TF-IDF) 
#gives a measure that takes the importance of a word
#into consideration depending on how frequently it occurs in a document and a corpus.

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 50000)

In [30]:
text_tfidf = list(textlemm)
text_tfidf = tv.fit_transform(textlemm).toarray()

In [31]:
#text_1 = list(text)
#text_1 = tv.fit_transform(text).toarray()
#print(text_1)
#text_2= tv.fit_transform(text)
#feature_names= tv.get_feature_names()
#print(text_2)

#feature_names= tv.get_feature_names()
#feature_names1= tv.get_feature_names()
# 0: 394 are the numerical values 
# we need text only as part of NLP
#print(feature_names1[394:500])

#matrix= vectors.todense()
#list_dense = matrix.tolist()
#df1= pd.DataFrame(text_1)
#print(df1)

Params -
* size: The number of dimensions of the embeddings and the default is 100.
* window: The maximum distance between a target word and words around the target word. The default window is 5.
* min_count: The minimum count of words to consider when training the model; words with occurrence less than this count will be ignored. The default for min_count is 5.
* workers: The number of partitions during training and the default workers is 3.
* sg: The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW.

In [32]:
features=text_tfidf
labels = is_sarcastic

In [33]:
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.1, random_state = 0)

In [34]:
labels_train, features_train

(13831    1
 18506    1
 24970    0
 19892    0
 2733     0
         ..
 13123    0
 19648    0
 9845     1
 10799    0
 2732     0
 Name: is_sarcastic, Length: 24038, dtype: int64,
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]))

In [35]:
##Choosing the Right Alogorthm:

In [36]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB

In [37]:
#lsvc = LinearSVC()
# training the model
#lsvc.fit(features_train, labels_train)
# getting the score of train and test data
#print(lsvc.score(features_train, labels_train)) # 90.93
#print(lsvc.score(features_test, labels_test))   # 83.75
   

In [38]:
# model 2:-
# Using Gaussuan Naive Bayes
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print(gnb.score(features_train, labels_train))  # 78.86
print(gnb.score(features_test, labels_test)) 

0.9128463266494716
0.7147135904155747


# We will use Word2Vec to get vectors which represents words in Multi-dimensional space (ex: 30d or 32d)
import gensim
from gensim.models import Word2Vec

In [ ]:
data['w2v']= data['textlemm'].apply(lambda x: word_tokenize(x))

In [ ]:
Word2Vec_1 = Word2Vec(data['w2v'], 
                        min_count= 2,
                        window=4,
                        size=500,
                        alpha=0.03, 
                        min_alpha=0.0007, 
                        sg = 1)

In [ ]:
Word2Vec_1.wv.vect

In [ ]:
Word2Vec_1 = Word2Vec(data['w2v'], 
                        min_count= 2,
                        window=4,
                        size=300, 
                        alpha=0.03, 
                        min_alpha=0.0007, 
                        sg = 1)
w2v_model.build_vocab(w2v_df, progress_per=10000)
w2v_model.train(w2v_df, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)
    return w2v_model


In [ ]:
Word2Vec_1.wv.vector_size

# Word2Vec_1.vector_size -- also works the same way

# Another method of dealing with the data set:

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json

In [ ]:
with open("/tmp/sarcasm.json","r") as f:
  datastore=json.load(f)
sentences = []
labels=[]
url[]

In [ ]:
datastore[1:5]

[{'article_link': 'https://www.huffingtonpost.com/entry/roseanne-revival-review_us_5ab3a497e4b054d118e04365',
  'headline': "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
  'is_sarcastic': 0},
 {'article_link': 'https://local.theonion.com/mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697',
  'headline': "mom starting to fear son's web series closest thing she will have to grandchild",
  'is_sarcastic': 1},
 {'article_link': 'https://politics.theonion.com/boehner-just-wants-wife-to-listen-not-come-up-with-alt-1819574302',
  'headline': 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
  'is_sarcastic': 1},
 {'article_link': 'https://www.huffingtonpost.com/entry/jk-rowling-wishes-snape-happy-birthday_us_569117c4e4b0cad15e64fdcb',
  'headline': 'j.k. rowling wishes snape happy birthday in the most magical way',
  'is_sarcastic': 0}]

**Notes on Tokenization using Keras(Tensor Flow)**:
- When dealing with blobs(images) we have pixel values which are numbers(gray scale encoded). But, whilst dealing with text which is usually in raw format, has to be encoded, so that it can be easily preprocessed. we dont have a generic framework to do that, instead we have built in function in keras to do that. encding is a higher side of processing because of higher chunks of data(MB and GB).Whenever we do some relatime application (lstm classification, basically 'coz of complexities of data).
- When we start to label for each of the word, to look at the similarity between words it becomes difficult, here the technique of bigrams, trigrams and ngrams. 
- this is where the keras (tensor flow) comes in which has an easier method.
- Using tokenization in keras, we can labels each word and provide a dictionary of the words used in the sentences to the model.
- Function used is **fit_on_texts()**

In [ ]:
pip install tensorflow 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence

In [ ]:
#demo
sentence = "Rahul is a very good boy"
res = text_to_word_sequence(sentence)
print(res)

['rahul', 'is', 'a', 'very', 'good', 'boy']


In [ ]:
t = Tokenizer()
t.fit_on_texts(sentence)

In [ ]:
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)

OrderedDict([('r', 2), ('a', 2), ('h', 1), ('u', 1), ('l', 1), ('i', 1), ('s', 1), ('v', 1), ('e', 1), ('y', 2), ('g', 1), ('o', 3), ('d', 1), ('b', 1)])
24
{'o': 1, 'r': 2, 'a': 3, 'y': 4, 'h': 5, 'u': 6, 'l': 7, 'i': 8, 's': 9, 'v': 10, 'e': 11, 'g': 12, 'd': 13, 'b': 14}
defaultdict(<class 'int'>, {'r': 2, 'a': 2, 'h': 1, 'u': 1, 'l': 1, 'i': 1, 's': 1, 'v': 1, 'e': 1, 'y': 2, 'g': 1, 'o': 3, 'd': 1, 'b': 1})
